### Load Pretrained Model
Load a pretrained Mamba Model that is compatible with Transformers Library

In [ ]:
from modeling_mamba import MambaForCausalLM
from transformers import AutoTokenizer

model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
tokenizer = AutoTokenizer.from_pretrained("Q-bert/Mamba-130M")

text = "Hi"

input_ids = tokenizer.encode(text, return_tensors="pt")

output = model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

In [ ]:
print(type(model))


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print("plain", print_trainable_parameters(model))

### Load LoRA adapters from the hub

In [ ]:
a1 = MambaForCausalLM.from_pretrained("mlsquare/exp-lora-ada-1")
a2 = MambaForCausalLM.from_pretrained("mlsquare/exp-lora-ada-2")

In [ ]:
a1.save_pretrained("./mbins/tmp/ada-1")
a2.save_pretrained("./mbins/tmp/ada-2")

### Average the adapters based on some weight
do weighed mean of the adapters of the same type

In [ ]:
from peft import PeftMixedModel, PeftModel

base_model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
print("base mamba", print_trainable_parameters(base_model))
peft_model = PeftModel.from_pretrained(base_model, "./mbins/tmp/ada-1", "adapter1")
print("base mamba", print_trainable_parameters(base_model))

# note this is just a copy of adapter 1 (only syntax verification)
peft_model.load_adapter("./mbins/tmp/ada-1", "adapter2")
print("two adapters loaded", print_trainable_parameters(peft_model))

# note this is just a copy of adapter 1 (only syntax verification)
peft_model.load_adapter("./mbins/tmp/ada-1", "adapter3")
print("two adapters loaded", print_trainable_parameters(peft_model))


peft_model.add_weighted_adapter(
    ["adapter1", "adapter2", "adapter3"],
    [1.0, 1.0, 1.0],
    combination_type="linear",
    adapter_name="merged",
)
print("lora added", print_trainable_parameters(peft_model))

peft_model.delete_adapter("adapter1")
peft_model.delete_adapter("adapter2")
peft_model.delete_adapter("adapter3")

peft_model.merge_and_unload()
print("merged mamba", print_trainable_parameters(peft_model))

In [ ]:
text = "Hi"
input_ids = tokenizer.encode(text, return_tensors="pt")
output = peft_model.generate(input_ids=input_ids)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)